# Pyomo Formulation of Learning With Errors (LWE) Problem

## Formulation Overview

## Import Statements

In [1]:
# Pyomo Import Statements

%pip install pyomo
import pyomo
import pyomo.environ as pe
from pyomo.opt import SolverFactory

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Too Many Hints Library Import Statements
from lwe_with_hints import *
import numpy as np
import time
import math

## Set Up LWE Instance

In [3]:
n = 5      # Number of Rows In Matrix
m = 10      # Number of Columns In Matrix
q = 521     # Prime Number For Finite Field Calculations  
eta = 10    # Binomial Distribution Parameter 

A,b,q,s,e = lwe_gen.generateToyInstance(n,m,q,eta)

# See Print Statements for Readability
print("A Matrix, shape:", A.shape, ": \n", A)
print("b vector, shape:", b.shape, ": \n", b)
print("s (secret) vector, shape:", s.shape, ": \n", s)
print("e (error) vector, shape:",e.shape, ": \n", e)



A Matrix, shape: (5, 10) : 
 [[295 210 282 442 444 303 342 116 255 254]
 [278 433 338 122 386 508 140 104 163 115]
 [468  22 439 196  79 447 383 341  95 188]
 [195 426 403 137 230  10 294 156 499 134]
 [464  51  20 162 349 314 450 106 480 195]]
b vector, shape: (10,) : 
 [485 371 488 130 475 136 306  88 432 128]
s (secret) vector, shape: (5,) : 
 [-3 -2 -1  1 -2]
e (error) vector, shape: (10,) : 
 [ 1  2  2 -1  0  0 -4 -1 -5  1]


In [ ]:
# The 1-Norm of e
sum_i = 0
for i in range(m):
    sum_i += abs(e[i])
print("1-Norm of e: ", sum_i)

1-Norm of e:  17


## Prepare LWE Instance for Formulation

In [ ]:
# Partition A Matrix and b vector
# Assumes Normality of A Matrix

A_0 = A[:,:n]                 # Creates the Square Submatrix of A
A_1 = A[:,n:]                 # Remaining Columns of A

b_0 = b[:n] 
b_1 = b[n:]

# Verify shapes
print("A_0 Matrix Shape: ", A_0.shape)
print("A_1 Matrix Shape: ", A_1.shape)
print("b_0 Matrix Shape: ", b_0.shape)
print("b_1 Matrix Shape: ", b_1.shape)

A_0 Matrix Shape:  (5, 5)
A_1 Matrix Shape:  (5, 5)
b_0 Matrix Shape:  (5,)
b_1 Matrix Shape:  (5,)


### Find the Inverse of A_0

In [6]:
# extended euclidean algo to find mod inverse of a in Z_q

def mod_inverse(a, q):
    t, new_t = 0, 1
    r, new_r = q, a

    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r

    if r > 1:
        raise ValueError(f"{a} has no modular inverse under modulo {q}")
    if t < 0:
        t = t + q

    return t

# inverts matrix in Z_q via gaussian elimination

def invert_matrix_mod_q(matrix, q):
    n = matrix.shape[0]
    A = np.copy(matrix)
    I = np.eye(n, dtype=int)
    
    # Gaussian elimination
    for i in range(n):
        factor = mod_inverse(A[i, i], q)
        A[i, :] = (A[i, :] * factor) % q
        I[i, :] = (I[i, :] * factor) % q

        for j in range(n):
            if i != j:
                factor = A[j, i]
                A[j, :] = (A[j, :] - factor * A[i, :]) % q
                I[j, :] = (I[j, :] - factor * I[i, :]) % q

    return I


# sanity check function to ensure matrix inversion worked in finite field

def confirm_inverse(matrix, inverse_matrix, q):
    n = matrix.shape[0]
    identity_matrix = np.eye(n, dtype=int)
    product = np.dot(matrix, inverse_matrix) % q
    return np.array_equal(product, identity_matrix)


In [ ]:
# Find inverse of A_0
A_0_inv = invert_matrix_mod_q(A_0,q)

print("A_0_inv Matrix, shape:", A_0_inv.shape, ": \n", A_0_inv)
print("Inversion Checker: ", confirm_inverse(A_0,A_0_inv,q))

A_0_inv Matrix, shape: (5, 5) : 
 [[228 202 379 152 459]
 [316 274 223 306 146]
 [181 397  16 243  79]
 [296 150 480 269 233]
 [112  20 449 116  35]]
Inversion Checker:  True


### Phi Function

In [8]:
# maps vectors in R^n to R^{m-n}

def phi(v,A_0_inv,A_1,b_1,b_0,q):
    tmp = np.dot(v,A_0_inv) % q
    tmp = np.dot(tmp,A_1) % q
    tmp = (tmp + b_1) % q
    tmp_1 = np.dot(b_0,A_0_inv) % q
    tmp_1 = np.dot(tmp_1,A_1) % q
    res = (tmp - tmp_1) % q
    return  res

### Creating W Matrix and u Vector

In [ ]:
# Prepare the W Matrix, vector u, and select natural number t

W = np.empty((n,(m-n)))         
iden_matrix = np.eye(n, dtype=int)     

# create u vector
zero_n = np.zeros(n, dtype=int)     
u = phi(zero_n,A_0_inv,A_1,b_1,b_0,q) 

# iterate over basis vectors and create entries of W
for i in range(n):
    eps_i = iden_matrix[:,i]        # ith basis vector
    phi_eps_i = phi(eps_i,A_0_inv,A_1,b_1,b_0,q)
    for j in range(m-n):
        psi_eps_i = (phi_eps_i - u) % q
        W[i,j] = psi_eps_i[j]

# probalistic bound for e
t = int(3*math.sqrt(eta))

print("W Matrix, shape:", W.shape, ": \n", W)
print("u vector, shape:", u.shape, ": \n", u)
print("Probalistic Bound on absolute of e: ", t)

W Matrix, shape: (5, 5) : 
 [[145. 407.  23. 187. 204.]
 [ 69. 402. 175. 333. 184.]
 [189. 320. 444. 122. 371.]
 [425. 268. 501. 330.   8.]
 [ 96. 341. 343. 153. 506.]]
u vector, shape: (5,) : 
 [285 497 281 270 258]
Probalistic Bound on absolute of e:  9


In [ ]:
# small change, convert W and u to dictionary for model import

dict_W = {(i+1, j+1): int(W[i, j]) for i in range(W.shape[0]) for j in range(W.shape[1])}
dict_u = {i+1: int(u[i]) for i in range(u.shape[0])}

print("Dictionary version of W: ", dict_W)
print("Dictionary version of u: ", dict_u)

Dictionary version of W:  {(1, 1): 145, (1, 2): 407, (1, 3): 23, (1, 4): 187, (1, 5): 204, (2, 1): 69, (2, 2): 402, (2, 3): 175, (2, 4): 333, (2, 5): 184, (3, 1): 189, (3, 2): 320, (3, 3): 444, (3, 4): 122, (3, 5): 371, (4, 1): 425, (4, 2): 268, (4, 3): 501, (4, 4): 330, (4, 5): 8, (5, 1): 96, (5, 2): 341, (5, 3): 343, (5, 4): 153, (5, 5): 506}
Dictionary version of u:  {1: 285, 2: 497, 3: 281, 4: 270, 5: 258}


In [11]:
y_con_low = -(t*(n*q - n + 1) + q - 1)
y_con_high = (t*(n*q - n + 1))

print("y_con_low: ", y_con_low)
print("y_con_high: ", y_con_high)

y_con_low:  -23929
y_con_high:  23409


## Model Formulation

### Model with Cutoff Constraint (Bounded Objective Function)

In [12]:
solver = pe.SolverFactory('cbc')
solver.options['maxSo'] = 2
solver.options['cutoff'] = 2*int(math.sqrt(m))*eta

print(2*int(math.sqrt(m))*eta)

60


In [ ]:
model = pe.ConcreteModel()
model.M = pe.RangeSet(1, m)
model.N = pe.RangeSet(1,n)
model.mn = pe.RangeSet(1, (m-n))

model.u_param_vec = pe.Param(model.mn, initialize=0, mutable=True)
model.W_param_matrix = pe.Param(model.N, model.mn, initialize=0, mutable=True)

for i in range(n):
    for j in range(m-n):
        model.u_param_vec[j+1] = int(u[j])
        model.W_param_matrix[i+1, j+1] = int(W[i, j])

model.X = pe.Var(model.M, within=pe.Integers)
model.Y = pe.Var(model.mn, within=pe.Integers)
model.a = pe.Var(model.M, within=pe.NonNegativeIntegers)
model.b = pe.Var(model.M, within=pe.NonNegativeIntegers)

In [14]:
model.objective = pe.Objective(
    expr=sum((model.a[i] + model.b[i]) for i in model.M),
    sense=pe.minimize
)

In [15]:
model.con_W_u = pe.Constraint(model.mn, 
    rule=lambda model, i: (( sum((model.W_param_matrix[j,i] * model.X[j]) for j in model.N) - model.X[n+i] + (q * model.Y[i])) == -model.u_param_vec[i]))

model.linearize_obj = pe.Constraint(model.M, 
    rule=lambda model, i: model.a[i] - model.b[i] == model.X[i])

model.max_X = pe.Constraint(model.M, 
    rule=lambda model, i: model.X[i] <= t)

model.min_X = pe.Constraint(model.M, 
    rule=lambda model, i: model.X[i] >= -t)

model.max_Y = pe.Constraint(model.mn, 
    rule=lambda model, i: model.Y[i] <= y_con_high)

model.min_Y = pe.Constraint(model.mn, 
    rule=lambda model, i: model.Y[i] >= y_con_low)

In [16]:
instance = model.create_instance()
#instance.pprint()

In [17]:
results = solver.solve(model, tee = True, keepfiles = True)

Solver log file: '/var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmpkt0sn2j9.cbc.log'
Solver solution file: '/var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmpscloxhy7.pyomo.soln'
Solver problem files: ('/var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmpscloxhy7.pyomo.lp',)
Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - /opt/homebrew/bin/cbc -maxSo 2 -cutoff 60 -printingOptions all -import /var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmpscloxhy7.pyomo.lp -stat=1 -solve -solu /var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmpscloxhy7.pyomo.soln (default strategy 1)
maxSolutions was changed from 2147483647 to 2
cutoff was changed from 1e+100 to 60
Option for printingOptions changed from normal to all
Presolve is modifying 30 integer bounds and re-presolving
Presolve 15 (-30) rows, 35 (0) columns and 65 (-30) elements
Statistics for presolved model
Original problem has 35 integers (0 of which binary)
==== 15 zero objective 2 differen

In [18]:
# Turn Solver Results into array

x_values = np.zeros(len(model.M))
for i in model.M:
    x_values[i-1] = pe.value(model.X[i])

In [19]:
# Print out results

print("IP Solver Result for error vector e: ", x_values)
print("Actual error vector e: ", e)

IP Solver Result for error vector e:  [ 1.  2.  2. -1.  0.  0. -4. -1. -5.  1.]
Actual error vector e:  [ 1  2  2 -1  0  0 -4 -1 -5  1]


In [20]:
# Using solver's prediction for the error vector, find s and compare with original

print("IP Solver Result for secret vector s: ", (np.dot( ((b_0 - x_values[:n]) % q), A_0_inv) % q) )
print("Original secret vector s (in mod q space): ", s % q)

IP Solver Result for secret vector s:  [518. 519. 520.   1. 519.]
Original secret vector s (in mod q space):  [518 519 520   1 519]


### Model without Cutoff Constraint (Unbounded Objective Function)

In [21]:
solver.options['cutoff'] = 1e50     # Default cbc solver cutoff value

In [22]:
results = solver.solve(model, tee = True, keepfiles = True)

Solver log file: '/var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmpt8c8isit.cbc.log'
Solver solution file: '/var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmp5_mrl5vi.pyomo.soln'
Solver problem files: ('/var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmp5_mrl5vi.pyomo.lp',)
Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - /opt/homebrew/bin/cbc -maxSo 2 -cutoff 1e+50 -printingOptions all -import /var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmp5_mrl5vi.pyomo.lp -stat=1 -solve -solu /var/folders/jh/gbzy3b7d3p92kqxrjmhg0h4w0000gn/T/tmp5_mrl5vi.pyomo.soln (default strategy 1)
maxSolutions was changed from 2147483647 to 2
cutoff was changed from 1e+100 to 1e+50
Option for printingOptions changed from normal to all
Presolve is modifying 30 integer bounds and re-presolving
Presolve 15 (-30) rows, 35 (0) columns and 65 (-30) elements
Statistics for presolved model
Original problem has 35 integers (0 of which binary)
==== 15 zero objective 2 di

In [ ]:
x_values = np.zeros(len(model.M))
for i in model.M:
    x_values[i-1] = pe.value(model.X[i])

In [ ]:
print("IP Solver Result for error vector e: ", x_values)
print("Actual error vector e: ", e)

IP Solver Result for error vector e:  [ 1.  2.  2. -1.  0.  0. -4. -1. -5.  1.]
Actual error vector e:  [ 1  2  2 -1  0  0 -4 -1 -5  1]


In [ ]:
# using solver's prediction for the error vector, find s and compare with original

print("IP Solver Result for secret vector s: ", (np.dot( ((b_0 - x_values[:n]) % q), A_0_inv) % q) )
print("Original secret vector s (in mod q space): ", s % q)

IP Solver Result for secret vector s:  [518. 519. 520.   1. 519.]
Original secret vector s (in mod q space):  [518 519 520   1 519]


## Lattice Reduction Algorithm Results (for comparison)

In [26]:
print("Starting lattice reduction.")
lattice = LWELattice(A,b,q)

start_time = time.time()
lattice.reduce()
end_time = time.time()

elapsed_time = end_time - start_time

print(f"The function took {elapsed_time} seconds to run.")

print("The LWE secret is:")
print(lattice.s)

print("Lattice reduction found the following vector:")
print(lattice.shortestVector)

Starting lattice reduction.
The function took 0.08894610404968262 seconds to run.
The LWE secret is:
[-3 -2 -1  1 -2]
Lattice reduction found the following vector:
[-1 -2 -2  1  0  0  4  1  5 -1 -3 -2 -1  1 -2 -1]
